In [1]:
import os

In [2]:
pwd

'/home/prem/My_Projects/End-to-End-Kidney-Disease-Classification/research'

In [3]:
os.chdir('../')

In [4]:
pwd

'/home/prem/My_Projects/End-to-End-Kidney-Disease-Classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_inlcude_top: bool
    params_weights: str
    params_classes: int  

In [8]:
from Kidney_Disease_Classification_App.utils import *
from Kidney_Disease_Classification_App.utils.common import read_yaml, create_directories

In [9]:
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [ ]:
class ConfigurationManager:
    
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)

        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_inlcude_top = self.params.INCLUDE_TOP, 
            params_weights = self.params.WEIGHTS, 
            params_classes = self.params.CLASSES
        )

        return prepare_base_model_config

In [10]:
import os 
import urllib.request as request
from zipfile import ZipFile 
import tensorflow as tf

2024-07-01 16:37:07.186836: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-01 16:37:07.215586: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 16:37:07.371650: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 16:37:07.373116: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 16:37:08.659531: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [ ]:
class PrepareBaseModel:
    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape= self.config.params_image_size, 
            weights=self.config.params_weights, 
            include_top=self.config.params_inlcude_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_base_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable=False
        elif (freeze_till is not None) and (freeze_till >0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False 
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layer.Dense(
            units=classes, 
            activation = "softmax"
        )(flatten_in)

        full_model = tf.keras.model.Model(
            inputs = model.input,
            outputs= prediction)

        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"])
        
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_base_model(
            model = self.model,
            classes = self.config.params_classes, 
            freeze_all= True, 
            freeze_till=None, 
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        


    @staticmethod
    def save_model(path: Path, model:tf.keras.Model):
        model.save(path)



In [ ]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e 